<center><h1 style="font-family: Tahoma, sans-serif;font-size:45px;margin:0 50px 0 50px">Vision Transformers but Simpler 😉</h1></center>
<img style="width:30%;" src="https://i.pinimg.com/originals/eb/be/01/ebbe012dc393a88f1bd5c80017836e61.png">


<center><h1 style="font-family: Tahoma, sans-serif;font-size:22px;">(Well not talking about transformers here but I am surely<br> trying to not bore you to death already, lol.)</h1></center>

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Importing Libraries</h1>

<h1 style="font-family: Tahoma, sans-serif;font-size:22px;">Lets quickly import the libraries that are needed for this model to work.</h1>

In [ ]:
!pip -q install vit-pytorch
!pip -q install linformer

In [ ]:
# Important for Printing stuff:
from __future__ import print_function

# Basic native Python Tool:
import glob
from itertools import chain
import os
import random
import zipfile

# Our all season best friends:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

# PyTorch because survival:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

# To keep time in check:
from tqdm.notebook import tqdm

# For Preprocessing of the Data:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

# For grabbing our pretrained Model:
from linformer import Linformer
from vit_pytorch.efficient import ViT

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Some Hyperparameters</h1>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px;">Let's define some of the values before hand right now so that later we know what we are working with.</h1>

In [ ]:
# Setting up some HyperParameters

batch_size = 64
epochs = 5
lr = 3e-5
gamma = 0.7
seed = 42

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Seeding</h1>

In [ ]:
# Defining a function to seed everything.
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONSEED"] = str(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# Running the function:
seed_everything(seed)

In [ ]:
# Setting the device as CUDA or moving stuff to the GPU:
device = "cuda"

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Data Preprocessing</h1><br>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px;float:left;">In this section we basically edit the annotated dataset <img src="https://i.pinimg.com/originals/c5/4c/c4/c54cc4fbfb8bc6834690a5eb96e0f523.jpg" style="width:30%; float: right;margin:-30px 50px 0 0"> 
    
that has been provided to us and form two seperate lists which have two things: Paths for Training Images and Labels.
In order to use those labels, I converted them from strings to numbers. I have no idea if that helped or not but again, you don't question your code if it is working just fine.</h1>


In [ ]:
# Defining the directories for our files/
train_dir = "../input/ranzcr-clip-catheter-line-classification/train"
test_dir = "../input/ranzcr-clip-catheter-line-classification/test"

In [ ]:
# Loading the data CSV.
data = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")
data.columns
data = data.drop(["data"], axis = 1)
data.head()

In [ ]:
# Converting the columns into integers.
ord_enc = OrdinalEncoder()
data[['label']] = ord_enc.fit_transform(data[['label']])

# Converting the Labels from floats to integers.
data.label = data.label.astype("int")

# Grabbing the labels as a list.
label = data["label"]
label = label.to_list()

In [ ]:
# Producing the list of paths for the trainingset image files:
train_list = []
for i in data.index:
    
    # Grabbing the file name.
    a = data["StudyInstanceUID"].loc[i]
    
    # Attaching the file's path to it.
    b = train_dir + "/" + a + ".jpg"
    
    # Puttting it in a tupple along with it's label.
    train_list.append((b, data['label'].loc[i]))

<h1 style="font-family: Tahoma, sans-serif;font-size:27px;"> If you upvoted the notebook without fully going through it, you will find $10,000 under your pillow tomorrow morning.</h1>

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Visualizing Data</h1><br>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px;float:left;">Let's check out our data.<h1>

In [ ]:
random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3,3, figsize=(16,12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx][0])
    ax.set_title(train_list[idx][1])
    ax.imshow(img)

<h1 style="font-family: Tahoma, sans-serif;font-size:22px;float:left;margin:-50px 50px 0 0">Splitting the data into training and validation datasets.</h1>

In [ ]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          random_state=seed)

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Data Augmentation</h1>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px;">Defining our transforms presets for data augmentation.</h1>


In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

valid_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

<h1 style="font-family: Tahoma, sans-serif;font-size:35px">PyTorch Dataset</h1>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px">Finally defining a PyTorch dataset, this one is as simple to understand as it could get.<br>Basically grab the transforms presets that we defined above. Other than that we have the file length function and the function that grabs the labels.

In [ ]:
class RANZCRDataset(Dataset):
    
    # Grabbing the transform presets.
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    # Returning the transformed images and the image's label.
    def __getitem__(self, idx):
        
        # Note that file list consists of tuples.
        # The first item in tuple is the image.
        img_path = self.file_list[idx][0]
        img = Image.open(img_path).convert("RGB")
        img_transformed = self.transform(img)
        
        # The second item in the tuple is the label.
        label = self.file_list[idx][1]
        
        return img_transformed, label
    

In [ ]:
# Defining the Datasets.
train_data = RANZCRDataset(train_list, transform = train_transforms)
valid_data = RANZCRDataset(valid_list, transform = valid_transforms)

In [ ]:
# Defining the Dataloaders.
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(dataset = valid_data, batch_size = batch_size, shuffle = True)

# Efficient Attention

In [ ]:
# Grabbing the transformer.
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [ ]:
# Grabbing the model.
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=11,
    transformer=efficient_transformer,
    channels=3,
).to(device)

In [ ]:
# Defining some other presets:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

<h1 style="font-family: Tahoma, sans-serif;font-size:27px;">I can't believe you came all over here. (That's what she said)</h1>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px;">An Upvote from you would probably get me a job, <br>please help me move out of my parent's house by hitting the upvote button! 😏</h1>

<h1 style="font-family: Tahoma, sans-serif;font-size:35px;">Training the Model.</h1>
<h1 style="font-family: Tahoma, sans-serif;font-size:22px"> Training the model Alas.</h1>

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    
    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)
        
        output = model(data)
        
        label = torch.nn.functional.one_hot(label, num_classes = 11)
        label = label.type_as(output)
        
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)
        
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)
        
            val_output = model(data)
            val_loss = criterion(val_output, label)
        
            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)
        
        
    print(
        f"Epoch: {epoch+1} - loss: {epoch_loss:.4f} - acc : {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy: .4f}\n"
    )